In [1]:
import os
os.chdir("D:/data")  # 切换到 D 盘的 data 目录
print(os.getcwd())

D:\data


In [10]:
import os
import pandas as pd

base_dir = "D:/DATA/SD_data"
loc_folders = [f"loc{i}" for i in range(1, 7)]
splits = ["train", "val"]

all_entries = []

for loc in loc_folders:
    for split in splits:
        csv_path = os.path.join(base_dir, loc, f"{split}.csv")
        if not os.path.exists(csv_path):
            print(f"Missing: {csv_path}")
            continue

        df = pd.read_csv(csv_path)

        # Calculate total number of cars and CVs
        df["car_total"] = df["car_left"] + df["car_right"]
        df["cv_total"] = df["cv_left"] + df["cv_right"]

        # Assign label based on vehicle type
        def classify(row):
            if row["car_total"] > 0 and row["cv_total"] == 0:
                return 0  # only car
            elif row["cv_total"] > 0 and row["car_total"] == 0:
                return 1  # only CV
            elif row["cv_total"] == 0 and row["car_total"] == 0:
                return 2  #none
            

        df["label"] = df.apply(classify, axis=1)
        df = df[df["label"].notna()]

        # Use relative path (from base_dir)
        df["rel_path"] = df["path"].apply(lambda x: os.path.join(loc, x.replace("/", os.sep)))

        df["loc"] = loc
        df["split"] = split

        all_entries.append(df[["rel_path", "label", "loc", "split"]])

# Combine all rows
all_data = pd.concat(all_entries, ignore_index=True)

# Save to unified CSV
output_csv = os.path.join(base_dir, "all_audio_labels.csv")
all_data.to_csv(output_csv, index=False)
print(f"Saved unified label file: {output_csv}")

Saved unified label file: D:/DATA/SD_data\all_audio_labels.csv


In [26]:
import os
import pandas as pd
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

def normalize_energy(energy_array):
    return (energy_array - np.min(energy_array)) / (np.max(energy_array) - np.min(energy_array) + 1e-8)

def process_flac_and_save_segments(row, output_base, sr=16000,
                                   segment_duration_vehicle=1.0,
                                   segment_duration_bg=1.0,
                                   baseline_secs=3, threshold_ratio=0.3):
    abs_path = os.path.join("D:/DATA/SD_data", row["rel_path"])  # ✅ abspath
    label = int(row["label"])
    loc = row["loc"]
    split = row["split"]

    try:
        y, _ = librosa.load(abs_path, sr=sr)
    except Exception as e:
        print(f"❌ Error loading {abs_path}: {e}")
        return []

    entries = []

    if label == 2:
        # === Background: fixed 1s segmentation ===
        seg_len = int(sr * segment_duration_bg)
        num_segs = len(y) // seg_len

        for i in range(num_segs):
            start = i * seg_len
            end = start + seg_len
            segment = y[start:end]
            if len(segment) < seg_len:
                continue

            filename = f"{loc}_{split}_{os.path.basename(abs_path).replace('.flac','')}_bg_{i+1:03d}.wav"
            out_dir = os.path.join(output_base, "background")
            os.makedirs(out_dir, exist_ok=True)
            out_path = os.path.join(out_dir, filename)
            sf.write(out_path, segment, sr)
            entries.append({"filepath": os.path.relpath(out_path, output_base), "label": label})

    elif label in [0, 1]:
        # === Car / CV: energy threshold filtering ===
        seg_len = int(sr * segment_duration_vehicle)
        num_segs = len(y) // seg_len
        energies = [np.sqrt(np.mean(y[i * seg_len:(i + 1) * seg_len] ** 2)) for i in range(num_segs)]
        energies = np.array(energies)
        energies_norm = normalize_energy(energies)
        baseline = np.mean(energies_norm[:baseline_secs])
        threshold = baseline + threshold_ratio
        smoothed = np.convolve(energies_norm, np.ones(3) / 3, mode='same')
        active = smoothed > threshold

        for i, is_active in enumerate(active):
            if not is_active:
                continue
            start = i * seg_len
            end = start + seg_len
            segment = y[start:end]
            if len(segment) < seg_len:
                continue

            class_dir = "car" if label == 0 else "cv"
            filename = f"{loc}_{split}_{os.path.basename(abs_path).replace('.flac','')}_seg2s_{i+1:03d}.wav"
            out_dir = os.path.join(output_base, class_dir)
            os.makedirs(out_dir, exist_ok=True)
            out_path = os.path.join(out_dir, filename)
            sf.write(out_path, segment, sr)
            entries.append({"filepath": os.path.relpath(out_path, output_base), "label": label})

    return entries

# ==== Main ====
input_csv = "D:/DATA/SD_data/all_audio_labels.csv"
base_dir = "D:/DATA/SD_data"
output_base = os.path.join(base_dir, "vehicle_segments")
output_csv = os.path.join(output_base, "vehicle_clips.csv")

df = pd.read_csv(input_csv)
df = df[df["label"].isin([0, 1, 2])]

all_entries = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    entries = process_flac_and_save_segments(row, output_base)
    all_entries.extend(entries)

df_out = pd.DataFrame(all_entries)
df_out.to_csv(output_csv, index=False)
print(f"\n✅ Saved segmented vehicle clips to {output_csv}")


100%|████████████████████████████████████████████████████████████████████████████| 10266/10266 [14:21<00:00, 11.91it/s]



✅ Saved segmented vehicle clips to D:/DATA/SD_data\vehicle_segments\vehicle_clips.csv


In [27]:
import pandas as pd

csv_path = "D:/DATA/SD_data/vehicle_segments/vehicle_clips.csv"
df = pd.read_csv(csv_path)

car_count = (df["label"] == 0).sum()
cv_count = (df["label"] == 1).sum()
bg_count = (df["label"] == 2).sum()
print(f"Number of car segments (label=0): {car_count}")
print(f"Number of cv segments (label=1):  {cv_count}")
print(f"Number of bg segments (label=2):  {bg_count}")


Number of car segments (label=0): 71313
Number of cv segments (label=1):  1329
Number of bg segments (label=2):  11400
